| Tables        | Are           | Cool  |
| ------------- |:-------------:| -----:|
| col 3 is      | right-aligned | $1600 |
| col 2 is      | centered      |   $12 |
| zebra stripes | are neat      |    $1 |



| Expt Num | Model         | F1 Score            |  epochs  |  lr   |   wordNgrams  |
| ---------|---------------|---------------------|----------|-------|---------------|
|    1     |  default      | 0.5525770000000000  |    25    |  1.0  |       2       |
|    2     |  default      | 0.5460660000000000  |   100    |  1.0  |       2       |
|    3     |  default      | 0.5498856582816073  |   100    |  0.1  |       2       |
|    4     |  default      | 0.5695227616558932  |    25    |  1.0  |       3       |
|    5     |  default      | 0.5772041248378066  |    25    |  1.0  |       4       |
|    6     |  default      | 0.5683045171871797  |    25    |  1.0  |       5       |
|    7     |  default      | 0.5483274021352313  |    25    |  1.0  |       6       |
|    8     |  default      | 0.5745956497490240  |    25    |  0.1  |       4       |
|    9     |  default      | 0.5726129216695254  |    25    |  0.01 |       4       |
|   10     |  default      | 0.5740194010965838  |   100    |  0.01 |       4       |
|   11     |  default      | 0.5741746761387380  |  1000    |  0.01 |       4       |
|   12     |  default      | 0.5744753571672705  |  1000    |  0.1  |       4       |
|   13     |  default      | 0.5692476963278779  |   100    |  1.0  |       4       |
|   14     |  default      | 0.5711939787889154  |  1000    |  1.0  |       4       |
|----------|---------------|---------------------|----------|-------|---------------|
|   15     |  losshs       | 0.5727887401683456  |   100    |  1.0  |       4       |
|   16     |  losshs       | 0.5655720560552185  |    25    |  1.0  |       4       |
|    x     |  losshs       | 0.5726968517380318  |  1000    |  1.0  |       4       |
|    x     |  losshs       |                     |    25    |  1.0  |               |
|    x     |  losshs       |                     |    25    |  1.0  |               |


In [1]:
# Imports
from sklearn.metrics import f1_score
from fastText import train_supervised
from pathlib import Path

In [2]:
# Set Paths
qiqc_expt_train_data = Path('//data/data1/datasets/kaggle/quora_insincere/output/qiqc_expt_train_set.txt')
qiqc_expt_test_data = Path('//data/data1/datasets/kaggle/quora_insincere/output/qiqc_expt_test_set.txt')
qiqc_expt_model_path = Path('/data/data1/datasets/kaggle/quora_insincere/models/qiqc_expt_fasttext_def.bin')
qiqc_expt_losshs_model_path = Path('/data/data1/datasets/kaggle/quora_insincere/models/qiqc_expt_fasttext_losshs.bin')

In [3]:
# Function to print results of the test
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

In [4]:
def train_fasttext_default(train_data, epochs=25, learning_rate=1.0, wordNgrams=2):
    # train_supervised uses the same arguments and defaults as the fastText cli
    model = train_supervised(
        input=train_data, epoch=epochs, lr=learning_rate, wordNgrams=wordNgrams, verbose=2, minCount=1
    )
    # print_results(*model.test(valid_data))
    return model

In [5]:
def train_fasttext_losshs(train_data, epochs=25, learning_rate=1.0, wordNgrams=2):
    model = train_supervised(
        input=train_data, epoch=epochs, lr=learning_rate, wordNgrams=wordNgrams, verbose=2, minCount=1,
        loss="hs"
    )
    # print_results(*model.test(valid_data))
    # model.save_model("cooking.bin")
    return model


In [6]:
def model_quantize(train_data, retrain=True, cutoff=100000):
    model.quantize(input=train_data, qnorm=True, retrain=retrain, cutoff=cutoff)
    # print_results(*model.test(valid_data))
    # model.save_model("cooking.ftz")
    return model

In [7]:
# Train
# model_default = train_fasttext_default(str(qiqc_expt_train_data.absolute()), epochs=1000, learning_rate=1.0, wordNgrams=4)
model_losshs = train_fasttext_losshs(str(qiqc_expt_train_data.absolute()), epochs=1000, learning_rate=1.0, wordNgrams=4)

In [8]:
# model_default.save_model(str(qiqc_expt_model_path))
model_losshs.save_model(str(qiqc_expt_losshs_model_path))

In [9]:
model_losshs.predict('my voice range is a2-c5 .  my chest voice goes up to f4 .  included sample in my higher chest range .  what is my voice typ')
                    

(('__label__0',), array([1.00001001]))

In [10]:
def predict_questions(model, expt_test_data):
    ''' From the test data, extract the question and predict using the trained model '''
    y_pred = []
    y_actual = []
    with qiqc_expt_test_data.open() as test_fh:
        for line in test_fh:
            if line.strip().startswith('qid'):
                continue
            # the label below is the actual label from the test split
            # The label format is: __label__0 or __label__1
            label = line.strip()[0:10]
            # Add the actual label 0 or 1 to y_actual
            y_actual.append(label[-1])
            
            question = line.strip()[11:]
            pred = model.predict(question)
            # The pred from abobe is of the format: (('__label__0',), array([1.00001001]))
            # So below shenanigans needs to be done to get to the actual 0 or 1 label
            y_pred.append(pred[0][0][-1])
    return y_actual, y_pred

In [11]:
y_actual, y_pred = predict_questions(model_losshs, qiqc_expt_test_data)

In [12]:
f1_score(y_actual, y_pred, pos_label='1', average='binary')

0.5726968517380318